In [47]:
import numpy as np
import pandas as pd
import sys
sys.path.append('/Users/diegofiori/Desktop/epfl/master_thesis/master_thesis/')
from utils import read_pickle
from tqdm.notebook import tqdm

In [466]:
path = '/Users/diegofiori/Desktop/epfl/master_thesis/results/'
sim_code = 'qs_no_comp'
density = read_pickle(path+f'physical_features_end_{sim_code}.pickle')
slices = read_pickle(path+f'slices_top_features_end_{sim_code}.pickle')

In [467]:
density = np.concatenate(density)

In [468]:
density.shape

(268,)

In [469]:
slices.shape

(5, 7680, 11134)

In [470]:
#slices = [slices[:, i:i+80] for i in range(0, slices.shape[1], 80)]

In [471]:
#slices = np.concatenate([np.expand_dims(slice_, axis=1) for slice_ in slices], axis=1)

In [472]:
slices = slices[:4]

In [473]:
indexes = []
arrays = []
for i in range(slices.shape[0]):
    for k in range(slices.shape[2]):
        arrays.append(slices[i, :, k].reshape((-1, 1)))
        indexes.append((i, k))
df = pd.DataFrame(data=np.concatenate(arrays, axis=1), columns=indexes)

In [474]:
df.shape

(7680, 44536)

In [475]:
df.head()

,"(0, 0)","(0, 1)","(0, 2)","(0, 3)","(0, 4)","(0, 5)","(0, 6)","(0, 7)","(0, 8)","(0, 9)",...,"(3, 11124)","(3, 11125)","(3, 11126)","(3, 11127)","(3, 11128)","(3, 11129)","(3, 11130)","(3, 11131)","(3, 11132)","(3, 11133)"
0,3.768583,4.062278,0.068191,0.186343,0.896838,1.270216,0.166997,0.279547,0.010298,0.021636,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.786159,3.978283,0.071347,0.185681,0.909932,1.251200,0.168637,0.283131,0.010680,0.021285,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.798708,3.928351,0.075165,0.182488,0.919788,1.239611,0.170250,0.283795,0.011001,0.020410,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.807307,3.837626,0.077927,0.183336,0.923947,1.236829,0.169759,0.289778,0.010817,0.021586,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.826311,3.953005,0.075382,0.180245,0.918134,1.298391,0.167393,0.291165,0.010460,0.021717,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [476]:
from pipeline_builder import get_pipeline_index

In [477]:
dict_transf = get_pipeline_index()
dict_transf

{'persistence_entropy': I[0, 2),
 'amplitude_bottleneck_inf': I[2, 4),
 'amplitude_wasserstein_1': I[4, 6),
 'amplitude_wasserstein_2': I[6, 8),
 'amplitude_landscape_1_1_100': I[8, 10),
 'amplitude_landscape_1_2_100': I[10, 12),
 'amplitude_landscape_2_1_100': I[12, 14),
 'amplitude_landscape_2_2_100': I[14, 16),
 'amplitude_betti_1_100': I[16, 18),
 'amplitude_betti_2_100': I[18, 20),
 'amplitude_heat_1_1.6_100': I[20, 22),
 'amplitude_heat_1_3.2_100': I[22, 24),
 'amplitude_heat_2_1.6_100': I[24, 26),
 'amplitude_heat_2_3.2_100': I[26, 28),
 'derivative_bottleneck_inf': I[28, 30),
 'derivative_wasserstein_1': I[30, 32),
 'derivative_wasserstein_2': I[32, 34),
 'heat_kernel_1.6': I[34, 5034),
 'heat_kernel_3.2': I[5034, 10034),
 'betti_curve': I[10034, 10134),
 'pers_landscape': I[10134, 11134)}

## Selecting features based on RandomForest

We firstly defined the regression problem.

In [478]:
import h5py

In [479]:
from scipy.io import loadmat
sim_code_k = sim_code
if sim_code.split('_')[-1] == 'comp':
    sim_code_k = sim_code.split('_')[0]
x = loadmat(f'/Users/diegofiori/Desktop/epfl/master_thesis/k_perp_end_{sim_code_k}.mat')
y = x['k_quantities']

In [480]:
y = np.concatenate([y[i, :, :] for i in range(y.shape[0])])

In [481]:
y.shape

(8000, 2)

In [482]:
y = y[:len(df)]

In [483]:
non_zero_ind = np.any(y>0, axis=1)

In [484]:
y1 = y[non_zero_ind, 0]
y2 = y[non_zero_ind, 1]

In [485]:
X = df.values[non_zero_ind]

In [486]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [487]:
from scipy.stats import pearsonr
def compute_correlation_with_old_features(target_feat, old_features_idx, X):
    corr = [pearsonr(target_feat, X[:, old_idx])[0] for old_idx in old_features_idx]
    return np.abs(corr)

In [ ]:
# remove some features which are higly correlated
X = np.nan_to_num(X)
columns_idx = []
columns_tuple_id = [(i, j) for i in range(slices.shape[0]) for j in range(slices.shape[2])]
equi_class = {}
class_id = 0
for i in tqdm(range(X.shape[1])):  
    feat_mat = X[:, i]
    if columns_idx:
        corrs = [compute_correlation_with_old_features(feat_mat, columns_idx, X)]
        if np.max(corrs) < 0.95:
            columns_idx.append(i)
            equi_class[class_id] = [columns_tuple_id[i]]
            class_id += 1
        else:
            equi_class[np.argmin(corrs)].append(columns_tuple_id[i])
    else:
        columns_idx.append(i)
        equi_class[class_id] = [columns_tuple_id[i]]
        class_id += 1
X = X[:, np.array(columns_idx)]

/Users/diegofiori/anaconda3/envs/giotto_tda_env/lib/python3.8/site-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [ ]:
equi_class = {equi_class[i][0]: equi_class[i] for i in equi_class.keys()}

In [ ]:
X.shape

In [ ]:
X_train, X_test, y1_train, y1_test, y2_train, y2_test = train_test_split(X, y1, y2, test_size=0.2)

In [ ]:
model1 = RandomForestRegressor(n_estimators=2000, n_jobs=-1)
model1.fit(X_train, y1_train)
model2 = RandomForestRegressor(n_estimators=2000, n_jobs=-1)
model2.fit(X_train, y2_train)

In [ ]:
score_1 = model1.score(X_test, y1_test)
score_2 = model2.score(X_test, y2_test)
total_score = score_1 + score_2

In [ ]:
score_2

In [ ]:
score_1

In [ ]:
total_score

In [ ]:
model1.predict(X_test[19:20])

In [ ]:
y1_test[19]

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y1_test, model1.predict(X_test))

In [ ]:
print('relative error')
mean_absolute_error(y1_test, model1.predict(X_test))/np.mean(y1_test)

In [ ]:
feature_importance = score_1/total_score*model1.feature_importances_ + score_2/total_score*model2.feature_importances_


In [ ]:
# group the features (we want to keep all the spatial dimensions)
n_features = slices.shape[2]
#feature_importance_temp = [feature_importance[i:i+n_features] for i in range(0, len(feature_importance), n_features)]
#feature_importance_temp = [np.mean(feature_importance_temp[i:i+80], axis=0) 
#                           for i in range(0, len(feature_importance_temp), 80)]

In [ ]:
feature_importance_idx = [(i, j) for i in range(slices.shape[0])
                          for j in range(n_features)]
feature_importance_idx = [feature_importance_idx[i] for i in range(len(feature_importance_idx)) if i in columns_idx]
feature_importance_idx_sort = sorted(feature_importance_idx, reverse=True,
                                key=lambda x: feature_importance[feature_importance_idx.index(x)])

In [ ]:
important_features_idx = []
threshold = 0.85
minimum_value = np.max(feature_importance)/100
for i, idx in enumerate(tqdm(feature_importance_idx_sort)):
    feat_value = feature_importance[feature_importance_idx.index(idx)]
    if feat_value < minimum_value:
        break
    feat_mat = X[:, feature_importance_idx.index(idx)]
    if important_features_idx:
        corrs = [compute_correlation_with_old_features(feat_mat, important_features_idx, X)]
        if np.max(corrs) < threshold:
            important_features_idx.append(i)
        else:
            list_temp = equi_class.pop(idx)
            equi_class[feature_importance_idx_sort[important_features_idx[np.argmax(corrs)]]] += list_temp
    else:
        important_features_idx.append(i)
important_features = [feature_importance_idx_sort[i] for i in important_features_idx]

In [ ]:
important_equi_class = {key: equi_class[key] for key in important_features}

In [ ]:
important_features

In [ ]:
len(important_features)

In [ ]:
from utils import write_pickle
write_pickle(path=path+f'selected_index_{sim_code}.pickle', array=important_features)

In [ ]:
write_pickle(path=path+f'selected_equi_class_{sim_code}.pickle', array=important_equi_class)

## Analysis for equi_class and indices

### Comparison with and without the the 2 big components

In [ ]:
sim_code = 'cs'

In [ ]:
equi_class_no_com = read_pickle(path=path+f'selected_equi_class_{sim_code}_no_comp.pickle')
equi_class_norm = read_pickle(path=path+f'selected_equi_class_{sim_code}.pickle')

In [ ]:
def intersect_equi_classes(equi_1, equi_2):
    common_eq_classes = {}
    class_idx = 0
    for key_1 in equi_1.keys():
        set_temp_1 = set(equi_1[key_1])
        for key_2 in equi_2.keys(): 
            set_res = set(equi_2[key_2]).intersection(set_temp_1)
            if len(set_res) > 0:
                common_eq_classes[class_idx] = list(set_res)
                class_idx += 1
    return common_eq_classes

In [ ]:
def union_equi_classes(equi_1, equi_2):
    union_eq_classes = {i: equi_2[key] for i, key in enumerate(equi_2.keys())}
    class_idx = len(union_eq_classes)
    for key_1 in equi_1.keys():
        set_temp_1 = set(equi_1[key_1])
        for key_2 in equi_2.keys(): 
            set_res = set(equi_2[key_2]).intersection(set_temp_1)
            if len(set_res) == 0:
                union_eq_classes[class_idx] = equi_1[key_1]
                class_idx += 1
                break
    return union_eq_classes

In [ ]:
intersection_equi = intersect_equi_classes(equi_class_no_com, equi_class_norm)

In [ ]:
union_equi = union_equi_classes(equi_class_no_com, equi_class_norm)

In [ ]:
len(intersection_equi)/len(union_equi)

In [47]:
cs_index = read_pickle(path+f'selected_index_cs.pickle')
sf_index = read_pickle(path+f'selected_index_sf.pickle')
ds_index = read_pickle(path+f'selected_index_ds.pickle')

In [44]:
len(set(cs_index).intersection(set(sf_index)))/len(set(cs_index).union(set(sf_index)))

0.2185792349726776

In [53]:
len(set(cs_index).intersection(set(sf_index)).intersection(set(ds_index)))

3

In [55]:
few_selected_indices = list(set(cs_index).intersection(set(sf_index)).intersection(set(ds_index)))

In [56]:
few_selected_indices

[(3, 0), (2, 6), (3, 4)]

In [47]:
len(cs_index)

88

In [48]:
len(sf_index)

135

In [49]:
cs_index[:10]

[(2, 0),
 (0, 10639),
 (0, 10185),
 (3, 11030),
 (2, 6494),
 (2, 10187),
 (2, 7),
 (3, 1),
 (1, 10385),
 (1, 4239)]

In [50]:
sf_index[:10]

[(2, 10639),
 (0, 10285),
 (0, 10235),
 (0, 10185),
 (2, 2968),
 (3, 10681),
 (3, 10224),
 (3, 2),
 (2, 2969),
 (3, 4)]

In [51]:
dict_transf = get_pipeline_index()
dict_transf

{'persistence_entropy': I[0, 2),
 'amplitude_bottleneck_inf': I[2, 4),
 'amplitude_wasserstein_1': I[4, 6),
 'amplitude_wasserstein_2': I[6, 8),
 'amplitude_landscape_1_1_100': I[8, 10),
 'amplitude_landscape_1_2_100': I[10, 12),
 'amplitude_landscape_2_1_100': I[12, 14),
 'amplitude_landscape_2_2_100': I[14, 16),
 'amplitude_betti_1_100': I[16, 18),
 'amplitude_betti_2_100': I[18, 20),
 'amplitude_heat_1_1.6_100': I[20, 22),
 'amplitude_heat_1_3.2_100': I[22, 24),
 'amplitude_heat_2_1.6_100': I[24, 26),
 'amplitude_heat_2_3.2_100': I[26, 28),
 'derivative_bottleneck_inf': I[28, 30),
 'derivative_wasserstein_1': I[30, 32),
 'derivative_wasserstein_2': I[32, 34),
 'heat_kernel_1.6': I[34, 5034),
 'heat_kernel_3.2': I[5034, 10034),
 'betti_curve': I[10034, 10134),
 'pers_landscape': I[10134, 11134)}

In [66]:
cs_index_no_comp = read_pickle(path+f'selected_index_cs_no_comp.pickle')

In [67]:
set(cs_index_no_comp).intersection(cs_index)

{(0, 10637),
 (1, 10236),
 (1, 10385),
 (1, 10637),
 (2, 6),
 (2, 7),
 (2, 10186),
 (2, 10236),
 (3, 0),
 (3, 1),
 (3, 4),
 (3, 5),
 (3, 7),
 (3, 30),
 (3, 2287),
 (3, 10230),
 (3, 10231),
 (3, 10731),
 (3, 10830),
 (3, 10881),
 (3, 11131)}